In [1]:
import azureml.core
import azureml.dataprep
from azureml.automl.core.constants import FeaturizationConfigMode
print(azureml.dataprep.__version__)
import azureml.core
print(azureml.core.VERSION)


1.1.33
1.0.79


In [2]:
from azureml.core import Workspace

ws = Workspace.from_config()

WARNING - Warning: Falling back to use azure cli login credentials.
If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [3]:
from azureml.core import Datastore

sql_datastore = Datastore.get(workspace=ws, datastore_name="ado_sql_datastore")


In [7]:
from azureml.core import Dataset
from azureml.data.datapath import DataPath

label ="MitigationScore"
query_string = 'SELECT MitigationDescription, MitigationScore FROM FeedbackItems'

query = DataPath(sql_datastore, query_string)
feedback_sql_ds = Dataset.Tabular.from_sql_query(query)

feedback_sql_ds.register(workspace=ws,
                         name="feedback_mitigation_score",
                         description = "Feedback from Azure DevOps",
                         create_new_version=True)

2020-01-13 08:13:30.709665 | ActivityCompleted: Activity=from_sql_query, HowEnded=Failure, Duration=636.93 [ms], Info = {'activity_id': 'c6e2619a-9af3-4181-a854-f5b79fb4b58a', 'activity_name': 'from_sql_query', 'activity_type': 'PublicApi', 'app_name': 'TabularDataset', 'source': 'azureml.dataset', 'version': '1.0.79'}, Exception=DatasetValidationError; Cannot load any data from the datastore using the SQL query "{}". Please make sure the datastore and query is correct.


DatasetValidationError: Cannot load any data from the datastore using the SQL query "{}". Please make sure the datastore and query is correct.

In [ ]:
def split_dataset(dataset):
    # Split the dataset into train and test datasets
    train_data, test_data = dataset.random_split(percentage=0.8, seed=223)

    # Register the train dataset with your workspace
    train_data.register(workspace = ws, 
                        name = 'mitigation_score_train_dataset',
                        description = 'Feedback from Azure DevOps training data',
                        create_new_version=True)

    # Register the test dataset with your workspace
    test_data.register(workspace = ws, 
                       name = 'mitigation_score_test_dataset', 
                       description = 'Feedback from Azure DevOps test data',
                       create_new_version=True)
    return train_data, test_data
    
train_data, test_data = split_dataset(feedback_sql_ds)    

In [ ]:
import logging
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(task = 'regression',
                             debug_log = 'automl_errors.log',
                             training_data = train_data,
                             label_column_name = label,
                             verbosity = logging.INFO,
                             enable_early_stopping = True,
                             max_concurrent_iterations = 4,
                             max_cores_per_iteration = -1,
                             n_cross_validations = 5,
                             primary_metric ='r2_score',
                             preprocess=True,
                             featurization = FeaturizationConfigMode.Auto,
                             enable_tf = True
                            )

In [ ]:
from azureml.core.experiment import Experiment
experiment = Experiment(ws, "ai-mitigation-score-experiment-dc-sql")

local_run = experiment.submit(automl_config, show_output=True)

In [ ]:
best_run, fitted_model = local_run.get_output()

model = best_run.register_model(model_name='best_sql_dc_mitigation_score_model', model_path='./outputs/model.pkl')

In [ ]:
print("Registered model:\n --> Name: {}\n --> Version: {}\n --> URL: {}".format(model.name, model.version, model.url))